In [1]:
import torch as t
from torch import nn,optim
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import time

import visdom

vis = visdom.Visdom(env = u'Front_side_face')



In [3]:
#有3高斯层，6,12,24
layers1 = 12
layers2 = 24
class CNN_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer0 = nn.Sequential(   #这一层只是进行尺寸变化。
                                    #gaussion_kernel(),
                                    nn.Conv2d(3,3,(5,5),stride=2,padding=0),     #这样可以保证输入120*120的图片时，输出为60*60
                                    #nn.ReLU(True)
                                    )
        self.layer1 = nn.Sequential(
                                    #gaussion_kernel0(),
                                    nn.Conv2d(3,layers1,(7,7),stride=1,padding=0),    #为了减少0填充对于结果的影响，不进行边缘填充，反正层数少。
                                    nn.BatchNorm2d(layers1),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(2,2)
                                    #nn.AdaptiveAvgPool2d(27)
                                    )
        
        self.layer2 = nn.Sequential(
                                    #gaus_layer(),
                                    #gaussion_kernel1(),
                                    nn.Conv2d(layers1,layers2,(5,5),stride=1,padding=0),
                                    nn.BatchNorm2d(layers2),
                                    nn.ReLU(True),
                                    nn.AvgPool2d(3,3)
                                    )
        
        
        #self.gaus_layer2 = gaus_layer()
        #self.conv3 = nn.Conv2d(12,24,(3,3),stride=1,padding=1)
        #self.pool2 = nn.AvgPool2d(2,2)
        self.full_layer1 = nn.Sequential(                   #因为输入是16*6*6
                                    nn.Linear(1176,64),     
                                    nn.BatchNorm1d(64),
                                    nn.ReLU(True)
                                    )
        self.full_layer2 = nn.Sequential(                  
                                    nn.Linear(64,32),     
                                    nn.BatchNorm1d(32),
                                    nn.ReLU(True)
                                    )
        self.full_layer3 = nn.Sequential(                  
                                    nn.Linear(32,16),     
                                    nn.BatchNorm1d(16),
                                    nn.ReLU(True)
                                    )
        self.full_layer_output = nn.Sequential(                   
                                    nn.Linear(16,2),     
                                    
                                    )
        #self.softmax = nn.Softmax()
        
    def forward(self,x):
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        #print("x.shape",x.shape)
        x = x.view(x.size()[0], -1)
        #print(type(x))
        #print("x.shape1",x.shape)
        x = self.full_layer1(x)
        x = self.full_layer2(x)
        x = self.full_layer3(x)
        x = self.full_layer_output(x)
        #x = self.softmax(x)
        
        return x
        

In [4]:
model = CNN_Net()

In [5]:
#加载模型必须在，前面定义模型之后，即在前面的model = ....之后才能执行这一行。
model.load_state_dict(t.load("C:\\Users\\SiChen\\face_sys\\front_side_face\\model_save\\Front_side_CNN_network_500_1e-4_3.12.24_4full-layers_NoGaussion_NoSoftmax.pth"))

In [ ]:
normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
transform = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize,
                                ])
test_dataset = ImageFolder("D:\\newfolder52",transform=transform)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [13]:
from PIL import Image
import shutil

front_wrong_face = "E:\\D_Continue\\newfolder1102\\front_wrong_face"          #保存错误的人脸的文件夹
side_wrong_face = "E:\\D_Continue\\newfolder1102\\side_wrong_face"
goal_folder = "E:\\D_Continue\\newfolder1102\\goal_folder"

model.cuda()
model.eval()
test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
transform = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize,
                                ])
test_dataset = ImageFolder("E:\\D_Continue\\newfolder2003")

for img0, label0 in test_dataset.imgs:            #!!!!!!!!!!!!!!!!!!!!!只有这样我才能得到需要的名字。
    #print(img0)
    #print(label0)
    img = Image.open(img0)                  #书本P108
    img = transform(img)         
    img = img.unsqueeze(0)                  #书本P109
    label = Variable(t.Tensor([label0]))
    label = label.long()
    
    img, label = Variable(img.float()).cuda(),label.cuda()
    out = model(img)
    #print(out.data)
    _,pred = t.max(out.cpu(),1)                 #
    num_correct = (pred==label.cpu()).sum()
    #print(num_correct)
    num_correct = num_correct.int()     #是torch.ByteTensor转为torch.IntTensor类型。size为1
    test_right_num = test_right_num + num_correct
    
    if (pred==label.cpu()).data[0] == 0:   #判断model处理后的结果是否正确。
        if not label0:            #因为clear_face对应的id是0.见前面的test_dataset.class_to_idx
            shutil.copy(img0,front_wrong_face)
            #print("F")
        else:
            #print("F")
            shutil.copy(img0,side_wrong_face)
    else:
        shutil.copy(img0,goal_folder)
    

acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)

Variable containing:
 642
[torch.IntTensor of size 1]

846
0.7588652482269503
